In [1]:
import pandas as pd
import numpy as np

In [2]:
def remove_blank(li):
    while li.count('') != 0:
        li.remove('')
    return li


tmp = open('data/aaindex/aaindex1')
aaindex = [_.replace('\n', '') for _ in tmp.readlines()]
tmp.close()

data_dic = {k:[] for k in 'ARNDCEQGHILKMFPSTWYV'}
title_list = []

for idx, line in enumerate(aaindex):
    
    # D : paper's title
    if line[0]=='D':
        title = line[2:]  # D paper_title -> indexing
        
        # check next line whether has 'R'(paper number)
        cnt = 1
        while aaindex[idx+cnt][0] != 'R':
            title += aaindex[idx+cnt]
            cnt += 1
        title_list.append(title)
        
    elif line[0] == 'I':
        # I : AA - value
        a = remove_blank(line[1:].split(' '))  # Amino Acid
        v1 = remove_blank(aaindex[idx+1].split(' '))  # value on first line
        v2 = remove_blank(aaindex[idx+2].split(' '))  # value on second line
        for ami, val1, val2 in zip(a, v1, v2):
            # a : D/L (so use 0, -1 index)
            data_dic[ami[0]].append(val1)
            data_dic[ami[-1]].append(val2)

df_aaindex = pd.DataFrame(data_dic, index=title_list)
print(df_aaindex.shape)

(566, 20)


* 결측치 처리

In [3]:
NA_idx = (df_aaindex=='NA').sum(axis=1)>=1
NA_index = list(df_aaindex.loc[NA_idx].index)
print('drop feature : ', len(NA_index))

df_aaindex = df_aaindex.drop(NA_index).astype(float)
print(df_aaindex.shape)

drop feature :  13
(553, 20)


* 중복 처리

In [4]:
tmp = {k:0 for k in df_aaindex.index.values}
for k in df_aaindex.index.values:
    tmp[k] += 1
check_title = [i for i, cnt in tmp.items() if cnt >= 2]
print('중복 개수 : ', len(check_title))

중복 개수 :  1


In [5]:
df_aaindex.loc[check_title]

,A,R,N,D,C,E,Q,G,H,I,L,K,M,F,P,S,T,W,Y,V
"Free energy in beta-strand region (Munoz-Serrano, 1994)",0.978,0.784,0.915,1.038,0.573,0.962,0.863,1.405,0.724,0.502,0.766,0.841,0.729,0.585,2.613,0.784,0.569,0.671,0.56,0.444
"Free energy in beta-strand region (Munoz-Serrano, 1994)",1.400,1.230,1.610,1.890,1.140,1.420,1.330,2.060,1.250,1.020,1.330,1.340,1.120,1.070,3.900,1.200,0.990,1.100,0.98,0.870


* 제거

In [6]:
df_aaindex.drop(check_title, inplace=True)

In [7]:
df_aaindex.shape

(551, 20)

In [8]:
df_aaindex.to_csv('data/aaindex/df_aaindex.csv', index=True)

# self select 19 index

In [9]:
aaindex19 = ['Average accessible surface area (Janin et al., 1978)',
       'Percentage of buried residues (Janin et al., 1978)',
       'Percentage of exposed residues (Janin et al., 1978)',
       'Hydrophobicity (Prabhakaran, 1990)',
       'Side chain hydropathy, uncorrected for solvation (Roseman, 1988)',
       'Modified Kyte-Doolittle hydrophobicity scale (Juretic et al., 1998)',
       'Number of hydrogen bond donors (Fauchere et al., 1988)',
       'Normalized frequency of reverse turn, with weights (Levitt, 1978)',
       'Normalized frequency of alpha-helix, unweighted (Levitt, 1978)',
       'Normalized frequency of beta-sheet, unweighted (Levitt, 1978)',
       'Distance between C-alpha and centroid of side chain (Levitt, 1976)',
       'Positive charge (Fauchere et al., 1988)',
       'Negative charge (Fauchere et al., 1988)',
       'Net charge (Klein et al., 1984)',
       'Electron-ion interaction potential values (Cosic, 1994)',
       'Isoelectric point (Zimmerman et al., 1968)',
       'Transfer free energy (Janin, 1979)',
       'Solvation free energy (Eisenberg-McLachlan, 1986)',
       'Partition energy (Guy, 1985)']
df_aaindex19 = df_aaindex.loc[aaindex19]

In [10]:
df_aaindex19.to_csv('data/aaindex/df_aaindex19.csv', index=True)